In [5]:
# first we define the simulator and setup a quantum instance

from qiskit import Aer
from qiskit.utils import QuantumInstance

backend = Aer.get_backend('statevector_simulator')
qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

In [13]:
# then we setup our ansatz for the wavefunction

from qiskit.circuit.library import RealAmplitudes

ansatz = RealAmplitudes(2, reps=1)
ansatz.draw()

┌──────────────────────────────────────┐
q_0: ┤0                                     ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3]) │
q_1: ┤1                                     ├
     └──────────────────────────────────────┘

In [7]:
# now we have to define the observable we want the minimum eigenvalue of
from qiskit.opflow import X,Y,Z,I

observable = Z^I

In [8]:
# set-up the vqe algorithm, with an optimizer and wave function
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA

vqe = VQE(ansatz=ansatz, optimizer=COBYLA(), quantum_instance=qinstance)

In [9]:
# we could run this for many observables
result = vqe.compute_minimum_eigenvalue(observable)
print(result.eigenvalue)

(-0.9999999745193989+0j)


In [23]:
# now we have to define the observable we want the minimum eigenvalue of
from qiskit.opflow import X,Y,Z,I,H

observable = Y^I

result = vqe.compute_minimum_eigenvalue(observable)
print(result.eigenvalue)

0j


## Lets try it for the HO, should get eigenvalue 0